<a href="https://colab.research.google.com/github/kirshed/FlightGear-WPF/blob/master/finalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MOUNTING TO DRIVE AND CD INTO FOLDER
from google.colab import drive
import numpy as np
import pandas as pd
import os

drive.mount('/content/drive')

# Enter the foldername in your Drive where you have saved the code and datasets.
# Recommended path: 'deep_perception/assignments/assignment1/'
FOLDERNAME = 'deep_perception/assignments/final_project'
ASSIGNMENTNAME = 'final_project'
os.chdir(f"drive/MyDrive/{FOLDERNAME}")
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/deep_perception/assignments/final_project


In [2]:
print(os.getcwd())

/content/drive/MyDrive/deep_perception/assignments/final_project


In [3]:
# !python main.py
!python main.py --resume --lr=0.01

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..
==> Resuming from checkpoint..
Traceback (most recent call last):
  File "main.py", line 81, in <module>
    net.load_state_dict(checkpoint['net'])
KeyError: 'net'


In [33]:
# retrieving saved net from drive
from vgg import *
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

net = VGG('VGG19')
device = 'cuda'
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
checkpoint = torch.load('./checkpoint/ckpt.pth')
net.load_state_dict(checkpoint['net'])
best_acc = checkpoint['acc']

In [ ]:
print(net)

In [34]:
# new gradcam class
class GradCamVGG(nn.Module):
    def __init__(self, net):
        super(GradCamVGG, self).__init__()

        self.vgg_model = net
        # cut the network and get the feture extractor: 
        #   all layers from the beginning to the last conv layer
        self.features_until_last_conv = self.vgg_model.module.features[:52] # layers [0 to 52) (excluding 52)
        # self.features_until_last_conv = self.vgg_model[:52]
        # create a new max pooling that links between features and classifier
        self.max_pool_layer = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        # get the classifier
        self.classifier = self.vgg_model.module.classifier
    
    def capture_gradients(self, grad):
        # hook for the gradients of the activations
        self.features_gradients = grad

    def extract_features(self, x):
        # extract features of kast conv layer
        return self.features_until_last_conv(x)

    def forward(self, x):
        # pass image trough the first part of the network to extract features
        x = self.extract_features(x)

        # hook the features tensor to extract features during backprop
        h = x.register_hook(self.capture_gradients)

        # forward through the rest of the network (max_pool_layer and classifier)
        x = self.max_pool_layer(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [35]:
# creating data loaders
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=1, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=False, num_workers=2)


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [36]:
def gradients_global_avg_polling(gradients):
  return torch.nn.AvgPool3d((1, gradients.shape[2], gradients.shape[3]))(gradients)

In [37]:
# creating heatmap
def grad_cam_heatmap(activations, alpha_values):
  activations = activations.clone()
  # activations = activations.reshape(1,1,1,-1)
  activations = torch.mul(activations, alpha_values)
  # create a heatmap by average the channels of the activations
  heatmap = torch.mean(activations, dim=1).squeeze()
  # apply relu and normalize the heatmap
  heatmap = F.relu(heatmap)
  heatmap /= torch.max(heatmap)
  return heatmap

In [84]:
# combining original image with heatmap
def get_image_with_heatmap(img, heatmap):
  # img = cv2.imread(content_image_path)
  # print(img.shape)
  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
  heatmap = np.uint8(255 * heatmap)
  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
  superimposed_img = heatmap * 0.4 + img
  return superimposed_img

In [79]:
# plotting combined image
def plot_superimposed_img(superimposed_img):
  # cv2_imshow(superimposed_img)
  plt.imshow(superimposed_img)
  plt.pause(0.001)

In [155]:
# interating over 5000 images and transforming them to new data (img+heatmap)
# saving new data and labels
from IPython.display import clear_output 
# creat an instance of our vgg model
vgg_grad_cam = GradCamVGG(net).cuda()

# set model in eval mode (alternative to freezing)
vgg_grad_cam.eval()

labels = []
for batch_idx, (inputs, targets) in enumerate(trainloader):
  if (batch_idx == 5000):
    break
  inputs, targets = inputs.to(device), targets.to(device)
  predictions = vgg_grad_cam(inputs)
  activations = vgg_grad_cam.extract_features(inputs)
  predictions[:, predictions.argmax(1)].backward()
  gradients = vgg_grad_cam.features_gradients
  # print("Last layer features shape:", activations.shape)
  # print("Last layer gradients shape:", gradients.shape)
  alpha_values = gradients_global_avg_polling(gradients)
  # print("Alpha values shape:", alpha_values.shape)
  # calculate heatmap
  heatmap = grad_cam_heatmap(activations, alpha_values).detach().cpu().numpy()
  # print heatmap shape
  # print("Heatmap Shape:", heatmap.shape)
  img = np.transpose(inputs[0].cpu().detach().numpy(), (1, 2, 0))
  heatmap_img = get_image_with_heatmap(img, heatmap)
  # plt.imshow(img)
  # plt.pause(0.001)
  plt.imshow(heatmap_img)
  plt.gca().set_axis_off()
  plt.savefig(f"./heatmap_imgs/{batch_idx}", bbox_inches='tight',transparent=True, pad_inches=0)
  labels.append(targets)
  plt.pause(0.001)
  clear_output()

In [168]:
# saving labels to drive
with open("./labels.txt", 'w+') as f:
  for lab in labels:
    f.write("%s\n" % lab.cpu().numpy()[0])

In [128]:
# functions for convert image to tensor for running net on changed images
from PIL import Image
from torchvision import transforms, models
def image_to_tensor(image_numpy, max_size=400, shape=None):
  
  # crop image if image is too big
  if max(image_numpy.size) > max_size:
    size = max_size
  else:
    size = max(image_numpy.size)
	
  size = (size, int(1.5*size))
  # if shape is given use it
  if shape is not None:
    size = shape
  
  # resize and normalize the image
  in_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])
  
  image = in_transform(image_numpy)[:3, :, :].unsqueeze(0)
  
  return image

def image_path_to_numpy(image_path):
  # load image into a numpy array from the given path
  return Image.open(image_path).convert('RGB') 

In [212]:
# dataset class for img_heatmap datatype
class HeatDataset(torch.utils.data.Dataset):
  # 'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        # 'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        # 'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        # 'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        # X = torch.load('data/' + ID + '.pt')
        X = image_path_to_numpy(f"./heatmap_imgs/{ID}.png")
        X = image_to_tensor(X)
        y = self.labels[index]

        return X, y

# loader examples
train_ids = list(range(0, 4000))
with open("./labels.txt", 'r') as f:
  labels = f.read().split('\n')
labels = labels[:-1]
# casting labels to ints 
for i in range(0, len(labels)):
    labels[i] = int(labels[i])

train_labs = labels[:4000]
test_ids = list(range(4000,5000))
test_labs = labels[4000:]

trainset = HeatDataset(train_ids, train_labs)
testset = HeatDataset(test_ids, test_labs)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)


In [213]:
print(len(test_labs))
print(len(test_ids))

1000
1000


In [214]:
# running the model on new data
# note that some of the code written in colab is written in new_main.py as well
!python new_main.py


Epoch: 0
 [================================================================>]  Step: 27ms | Tot: 2m1s | Loss: 2.589 | Acc: 9.300% (372/4000) 4000/4000 
 [================================================================>]  Step: 11ms | Tot: 10s295ms | Loss: 5.292 | Acc: 13.700% (137/1000) 1000/1000 
Saving..

Epoch: 1
 [================================================================>]  Step: 27ms | Tot: 2m1s | Loss: 2.329 | Acc: 9.425% (377/4000) 4000/4000 
 [================================================================>]  Step: 9ms | Tot: 10s432ms | Loss: 6.313 | Acc: 10.900% (109/1000) 1000/1000 

Epoch: 2
 [================================================================>]  Step: 27ms | Tot: 2m1s | Loss: 2.327 | Acc: 9.725% (389/4000) 4000/4000 
 [================================================================>]  Step: 8ms | Tot: 10s612ms | Loss: 9.386 | Acc: 11.700% (117/1000) 1000/1000 

Epoch: 3
 [================================================================>]  Step: 27ms